# Optional Lab - ฟังก์ชันซอฟต์แม็กซ์
ในห้องปฏิบัติการนี้ เราจะสำรวจฟังก์ชันซอฟต์แม็กซ์ ฟังก์ชันนี้ใช้ในทั้งการถดถอยซอฟต์แม็กซ์และในเครือข่ายประสาทเทียมเมื่อแก้ปัญหาการจำแนกประเภทหลายคลาส
<center>  <img  src="./images/C2_W2_Softmax_Header_v1.png" width="600" />  <center/>

  

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_common_C5.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/autils.py'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_softmax.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('autils.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url4)
with open('lab_utils_softmax.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

> **หมายเหตุ**: โดยปกติ ในหลักสูตรนี้ โน้ตบุ๊กจะใช้รูปแบบการเริ่มนับด้วย 0 และสิ้นสุดด้วย N-1 ซึ่งเขียนเป็น $\sum_{i=0}^{N-1}$, ในขณะที่การบรรยายจะเริ่มด้วย 1 และสิ้นสุดด้วย N ซึ่งเขียนเป็น  $\sum_{i=1}^{N}$. เนื่องจากโค้ดโดยทั่วไปจะเริ่มการวนซ้ำด้วย 0 ในขณะที่ในการบรรยาย การนับจาก 1 ถึง N จะทำให้สมการมีความชัดเจนและกระชับมากขึ้น โน้ตบุ๊กนี้มีสมการมากกว่าปกติสำหรับห้องปฏิบัติการ ดังนั้นจึงจะขัดกับรูปแบบนี้และจะนับจาก 1 ถึง N

## Softmax Function
ในทั้งการถดถอยซอฟต์แม็กซ์และเครือข่ายประสาทเทียมที่มีผลลัพธ์ซอฟต์แม็กซ์ จะมีการสร้างผลลัพธ์ N ผลลัพธ์และเลือกผลลัพธ์หนึ่งเป็นหมวดหมู่ที่ทำนายได้ ในทั้งสองกรณีจะมีการสร้างเวกเตอร์ $\mathbf{z}$ โดยใช้ฟังก์ชันเชิงเส้นซึ่งนำไปใช้กับฟังก์ชันซอฟต์แม็กซ์ ฟังก์ชันซอฟต์แม็กซ์จะแปลง $\mathbf{z}$  เป็นการแจกแจงความน่าจะเป็นตามที่อธิบายไว้ด้านล่าง หลังจากใช้ซอฟต์แม็กซ์ ผลลัพธ์แต่ละรายการจะอยู่ระหว่าง 0 ถึง 1 และผลลัพธ์จะบวกกันเป็น 1 เพื่อให้สามารถตีความได้ว่าเป็นความน่าจะเป็น อินพุตที่ใหญ่กว่าจะสอดคล้องกับความน่าจะเป็นของผลลัพธ์ที่ใหญ่กว่า
<center>  <img  src="./images/C2_W2_SoftmaxReg_NN.png" width="600" />  

The softmax สามารถเขียนได้ดังนี้:
$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$
The output $\mathbf{a}$ is a vector of length N, so for softmax regression, you could also write:
\begin{align}
\mathbf{a}(x) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w},b) \\
\vdots \\
P(y = N | \mathbf{x}; \mathbf{w},b)
\end{bmatrix}
=
\frac{1}{ \sum_{k=1}^{N}{e^{z_k} }}
\begin{bmatrix}
e^{z_1} \\
\vdots \\
e^{z_{N}} \\
\end{bmatrix} \tag{2}
\end{align}


ซึ่งแสดงให้เห็นว่าผลลัพธ์เป็นเวกเตอร์ของความน่าจะเป็น รายการแรกคือความน่าจะเป็นที่อินพุตเป็นหมวดหมู่แรกเมื่อกำหนดอินพุต $\mathbf{x}$ และพารามิเตอร์  $\mathbf{w}$ และ $\mathbf{b}$.  
มาสร้างการนำไปใช้ NumPy:

In [ ]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponenial
    sm = ez/np.sum(ez)
    return(sm)

ด้านล่าง ให้เปลี่ยนค่าของอินพุต `z` โดยใช้แถบเลื่อน

In [ ]:
plt.close("all")
plt_softmax(my_softmax)

เมื่อคุณกำลังเปลี่ยนค่าของ z's ด้านบน มีสิ่งสำคัญบางอย่างที่ควรทราบ:
* เอ็กซ์โปเนนเชียลในตัวเศษของซอฟต์แม็กซ์จะขยายความแตกต่างเล็กน้อยในค่า 
* ค่าผลลัพธ์รวมเป็นหนึ่ง
* ซอฟต์แม็กซ์ครอบคลุมเอาต์พุตทั้งหมด การเปลี่ยนแปลงใน  `z0` เช่น จะเปลี่ยนค่าของ `a0`-`a3`. เปรียบเทียบกับการกระตุ้นอื่นๆ เช่น ReLU หรือ Sigmoid ที่มีอินพุตเดียวและเอาต์พุตเดียว

## Cost
<center> <img  src="./images/C2_W2_SoftMaxCost_v1.png" width="400" />    <center/>

ฟังก์ชันการสูญเสียที่เกี่ยวข้องกับซอฟต์แม็กซ์ การสูญเสียข้ามเอนโทรปี คือ:\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} \tag{3}
\end{equation}

โดยที่ y คือหมวดหมู่เป้าหมายสำหรับตัวอย่างนี้ และ $\mathbf{a}$  คือผลลัพธ์ของฟังก์ชันซอฟต์แม็กซ์ โดยเฉพาะค่าใน  $\mathbf{a}$ เป็นความน่าจะเป็นที่รวมกันเป็นหนึ่ง
>**การทบทวน:** ในหลักสูตรนี้ การสูญเสีย (Loss) เป็นสำหรับตัวอย่างหนึ่ง ในขณะที่ต้นทุน (Cost) ครอบคลุมตัวอย่างทั้งหมด
 
 
โปรดทราบใน (3) ข้างต้น เฉพาะบรรทัดที่สอดคล้องกับเป้าหมายเท่านั้นที่มีส่วนทำให้เกิดการสูญเสีย บรรทัดอื่น ๆ มีค่าเป็นศูนย์ เพื่อเขียนสมการต้นทุน เราต้องการ 'ฟังก์ชันตัวบ่งชี้' ที่จะมีค่าเป็น 1 เมื่อดัชนีตรงกับเป้าหมายและเป็นศูนย์ในกรณีอื่น ๆ
    $$\mathbf{1}\{y == n\} = =\begin{cases}
    1, & \text{if $y==n$}.\\
    0, & \text{otherwise}.
  \end{cases}$$
ตอนนี้ cost คือ:
\begin{align}
J(\mathbf{w},b) = -\frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{N}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{k=1}^N e^{z^{(i)}_k} }\right] \tag{4}
\end{align}

โดยที่  $m$ คือจำนวนตัวอย่าง, $N$ คือจำนวนผลลัพธ์ นี่คือค่าเฉลี่ยของการสูญเสียทั้งหมด


## Tensorflow
ห้องปฏิบัติการนี้จะกล่าวถึงสองวิธีในการนำฟังก์ชันซอฟต์แม็กซ์และการสูญเสียแบบครอสเอนโทรปีไปใช้ใน TensorFlow ซึ่งเป็นวิธีที่ 'ชัดเจน' และวิธีที่ 'เป็นที่ต้องการ' วิธีแรกนั้นตรงไปตรงมาที่สุดในขณะที่วิธีหลังนั้นมีความเสถียรทางตัวเลขมากกว่า

มาเริ่มกันด้วยการสร้างชุดข้อมูลเพื่อฝึกโมเดลการจำแนกประเภทหลายคลาส

In [ ]:
# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

### The *Obvious* organization

โมเดลด้านล่างนี้ถูกนำไปใช้กับซอฟต์แม็กซ์เป็นฟังก์ชันการกระตุ้นในเลเยอร์สุดท้าย ฟังก์ชันการสูญเสียถูกระบุแยกกันในคำสั่ง `compile`  

ฟังก์ชันการสูญเสียคือ  `SparseCategoricalCrossentropy` การสูญเสียนี้มีการอธิบายใน (3) ข้างต้น ในโมเดลนี้ ซอฟต์แม็กซ์เกิดขึ้นในเลเยอร์สุดท้าย ฟังก์ชันการสูญเสียรับเอาผลลัพธ์ซอฟต์แม็กซ์ซึ่งเป็นเวกเตอร์ของความน่าจะเป็น

In [ ]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
        

เนื่องจากซอฟต์แม็กซ์ถูกรวมเข้ากับเลเยอร์เอาต์พุต ผลลัพธ์จึงเป็นเวกเตอร์ของความน่าจะเป็น

In [ ]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

### Preferred <img align="Right" src="./images/C2_W2_softmax_accurate_v1.png"  style=" width:400px; padding: 10px 20px ; ">
จากการบรรยายคุณอาจจำได้ว่าจะได้ผลลัพธ์ที่เสถียรและแม่นยำยิ่งขึ้นหากรวมซอฟต์แม็กซ์และการสูญเสียเข้าด้วยกันระหว่างการฝึกอบรม การจัดระเบียบ 'ที่ต้องการ' แสดงไว้ที่นี่

การแปลเป็นภาษาไทย
"ในองค์กรที่ต้องการ ชั้นสุดท้ายจะมีการกระตุ้นเชิงเส้น ผลลัพธ์ในรูปแบบนี้เรียกว่า logits เนื่องจากเหตุผลทางประวัติศาสตร์ ฟังก์ชันการสูญเสียมีอาร์กิวเมนต์เพิ่มเติม: `from_logits = True` ซึ่งแจ้งให้ฟังก์ชันการสูญเสียทราบว่าควรมีการรวมการดำเนินการซอฟต์แม็กซ์ไว้ในการคำนวณการสูญเสีย ซึ่งช่วยให้สามารถนำไปใช้ได้อย่างเหมาะสม

In [ ]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

#### การจัดการผลลัพธ์

โปรดสังเกตว่าในโมเดลที่ต้องการ ผลลัพธ์ไม่ได้เป็นความน่าจะเป็น แต่สามารถอยู่ในช่วงตั้งแต่จำนวนลบขนาดใหญ่ไปจนถึงจำนวนบวกขนาดใหญ่ ผลลัพธ์จะต้องถูกส่งผ่านซอฟต์แม็กซ์เมื่อทำการทำนายที่คาดว่าจะเป็นความน่าจะเป็น

มาดูผลลัพธ์ของโมเดลที่ต้องการกัน:

In [ ]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

ผลลัพธ์การทำนายไม่ใช่ความน่าจะเป็น!

หากต้องการให้ผลลัพธ์เป็นความน่าจะเป็น ผลลัพธ์ควรได้รับการประมวลผลโดย [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax).

In [ ]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

เพื่อเลือกหมวดหมู่ที่เป็นไปได้มากที่สุด ไม่จำเป็นต้องใช้ซอฟต์แม็กซ์ สามารถหาดัชนีของผลลัพธ์ที่ใหญ่ที่สุดโดยใช้ [np.argmax()](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

In [ ]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

## SparseCategorialCrossentropy หรือ CategoricalCrossEntropy
Tensorflow มีสองรูปแบบที่เป็นไปได้สำหรับค่าเป้าหมายและการเลือกการสูญเสียจะกำหนดรูปแบบที่คาดหวัง
- SparseCategorialCrossentropy: คาดหวังว่าเป้าหมายจะเป็นจำนวนเต็มที่สอดคล้องกับดัชนี ตัวอย่างเช่น หากมีค่าเป้าหมายที่เป็นไปได้ 10 ค่า y จะอยู่ระหว่าง 0 ถึง 9 
- CategoricalCrossEntropy:  คาดหวังว่าค่าเป้าหมายของตัวอย่างจะเป็นการเข้ารหัสแบบ one-hot โดยค่าที่ดัชนีเป้าหมายจะเป็น 1 ในขณะที่รายการ N-1 อื่น ๆ จะเป็นศูนย์ ตัวอย่างที่มีค่าเป้าหมายที่เป็นไปได้ 10 ค่า โดยเป้าหมายคือ 2 จะเป็น [0,0,1,0,0,0,0,0,0,0].


## ขอแสดงความยินดี!

ในห้องปฏิบัติการนี้ คุณได้
-ทำความคุ้นเคยกับฟังก์ชันซอฟต์แม็กซ์และการใช้งานในการถดถอยซอฟต์แม็กซ์และการกระตุ้นซอฟต์แม็กซ์ในเครือข่ายประสาทเทียม
- เรียนรู้การสร้างแบบจำลองที่เป็นที่นิยมใน TensorFlow:
    - ไม่ใช้การกระตุ้นในเลเยอร์สุดท้าย (เหมือนกับการกระตุ้นเชิงเส้น)
    - ฟังก์ชันการสูญเสีย SparseCategoricalCrossentropy
    - ใช้ from_logits=True 
- ตระหนักว่าไม่เหมือน ReLU และ Sigmoid ซอฟต์แม็กซ์ครอบคลุมหลายเอาต์พุต